## Scenario 2 -  Multi-tenancy

### **SupportWizard** - Support Analyis SaaS Platform

- Allow its users to sign up and upload their own customer support data
- The platform can then analyse the information to identify where they could improve their support schema, and even their own products 

### Solution 

Each end user will have their own isolated "space", to which they can uplaod data. Then, they can use SupportWizard dashboards / platform to see analyses of their own data. 


### Create the collection


In [15]:
import os
import weaviate
from weaviate.classes.config import Property, DataType, Configure

client = weaviate.connect_to_local(
    headers={"X-Cohere-Api-Key": os.getenv("COHERE_API_KEY")}
)

collection_name = "SupportChatMT"

# For re-running the demo only: Delete existing collection if it exists
client.collections.delete(collection_name)

# Create a new collection with specified properties and vectorizer configuration
chunks = client.collections.create(
    name=collection_name,
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="dialogue_id", data_type=DataType.INT),
        Property(name="company_author", data_type=DataType.TEXT),
        Property(name="created_at", data_type=DataType.DATE),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="text_with_metadata",
            source_properties=["text", "company_author"],
            vector_index_config=Configure.VectorIndex.hnsw(),
            api_endpoint="http://host.docker.internal:11434",
            model="nomic-embed-text",
        )
    ],
    generative_config=Configure.Generative.cohere(model="command-r-plus"),
    multi_tenancy_config=Configure.multi_tenancy(enabled=True, auto_tenant_creation=True)
)

In [16]:
import h5py
import json
import numpy as np


def get_hdf5_obj(file_path):
    with h5py.File(file_path, "r") as hf:
        for uuid in hf.keys():
            src_obj = hf[uuid]

            # Get the object properties
            properties = json.loads(src_obj["object"][()])

            # Get the vector(s)
            vectors = {}
            for key in src_obj.keys():
                if key.startswith("vector_"):
                    vector_name = key.split("_", 1)[1]
                    vectors[vector_name] = np.asarray(src_obj[key])

            yield uuid, properties, vectors

In [17]:
from tqdm import tqdm

with client.batch.fixed_size(batch_size=200) as batch:
    for uuid, properties, vectors in tqdm(get_hdf5_obj("data/twitter_customer_support_nomic.h5")):
        if properties["company_author"] is None or properties["company_author"].strip() == "":
            tenant_name = "unknown"
        else:
            tenant_name = properties["company_author"]
        batch.add_object(
            collection=collection_name,
            uuid=uuid,
            properties=properties,
            vector={"text_with_metadata": vectors["text_with_metadata"]},
            tenant=tenant_name
        )


50000it [00:18, 2659.57it/s]


In [18]:
print(f"Processed {len(client.batch.results.objs.all_responses)} objects.")

Processed 50000 objects.


/Users/jphwang/code/demo-code/202501-vectordb-patterns/.venv/lib/python3.10/site-packages/weaviate/warnings.py:213: DeprecationWarning: Dep020: The `all_responses` attribute in the `BatchResults` object is deprecated and will be removed by Q4 2024. Please instead use the `errors` and `uuids` attributes.
  warnings.warn(


In [19]:
if len(client.batch.failed_objects) > 0:
    print("*" * 80)
    print(f"***** Failed to add {len(client.batch.failed_objects)} objects *****")
    print("*" * 80)
    print(client.batch.failed_objects[:3])

In [20]:
support_chats = client.collections.get(collection_name)

In [ ]:
support_chats.tenants.get()


{'British_Airways': Tenant(name='British_Airways', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'GloCare': Tenant(name='GloCare', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'DunkinDonuts': Tenant(name='DunkinDonuts', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'AldiUK': Tenant(name='AldiUK', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'idea_cares': Tenant(name='idea_cares', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),
 'AskTarget': Tenant(name='AskTarget', activityStatusInternal=<TenantActivityStatus.ACTIVE: 'ACTIVE'>, activityStatus=<_TenantActivistatusServerValues.HOT: 'HOT'>),


In [27]:
example_tenant_names = [
    "Delta", "DellCares", "AmazonHelp", "AppleSupport", "SpotifyCares"
]

In [28]:
tenant_data = support_chats.with_tenant(example_tenant_names[0])

In [29]:
response = tenant_data.query.fetch_objects(limit=2, include_vector=True)

In [30]:
print(response.objects[0].uuid)

0005cca3-178c-5811-a756-f7d97249989d


In [31]:
for k, v in response.objects[0].properties.items():
    print(f"\n|| {k} || \n{v}")


|| text || 
User_256481: No more nonstop service from LAX-CVG except a red eye?? What are you people thinking? If I have to connect, you’ve lost a customer. Pls reconsider. You were the only nonstop left...
Delta: Scott, I am sorry for your disappointment in our schedule. We appreciate your feedback and I will send this over to our leadership team. *TBW

|| company_author || 
Delta

|| created_at || 
2017-12-03 11:01:33+00:00

|| dialogue_id || 
579527


In [32]:
for k, v in response.objects[0].vector.items():
    print(k, v[:3])

text_with_metadata [0.352222204208374, 0.7630466222763062, -3.3135671615600586]


### Queries

In [33]:
def display_objects(response):
    for o in response.objects:
        print(o.uuid, "\n")
        print(o.properties["text"][:200], "\n")

In [34]:
response = tenant_data.query.near_text("return process", limit=3)
display_objects(response)

3aeb3760-8d3e-5750-8aef-1953bd089c0d 

User_205654: if a flight booked with miles is cancelled within the cancellation window, how long does it take for miles to be refunded to my acct?
Delta: Hello, Robert. It should take about 24-48 hrs  

000894a8-57cf-55b8-8a23-455a12c0d4d5 

User_209651: Any chance delta will respond to my complaint and refund request. It's only been nearly a month. #greataftercare
Delta: Hello Willie, you can check the status of a refund at 1-8001847-057 

191d93e2-3c94-52f1-a9b1-458b5659e020 

User_195933: Kudos to @Delta for a thoughtful policy &amp; refunding my Puerto Rico flights. Disappointed in @SouthwestAir for not refunding, only voucher.
Delta: You're welcome, Malcolm. We do what w 



In [35]:
response = tenant_data.query.bm25("return process", limit=3)
display_objects(response)

34d433e1-37c8-5698-b5df-4794a48a16ec 

User_208765: I’m having trouble reaching you by 800 #, but need to cancel my return flight from Cancun today.  Family emergency!
Delta: I should be able to assist you with your cancellation. One momen 

08ae4975-bcc9-5bf9-9524-0d2fd5559970 

User_195931: won't reply. Flew Business class (Delta One) cabin from Detroit-Tokyo and return Tokyo-Detroit. Here's bedding. Disgusting! https://t.co/tVloZkQUFI
Delta: Oh, My! I'm deeply sorry about t 

09668a8d-e1d8-568e-b7de-d3faa30fb73e 

User_134888: I booked a roundtrip flight. If I miss my outbound, can I still take my return flight?
Delta: Of course, any change is subject to fare difference as well as any applicable change fees. *A 



In [36]:
response = tenant_data.query.hybrid("return process", limit=3)
display_objects(response)

2df70df5-e9cc-5c11-a395-c3f93d8cf2db 

User_304584: On return flight DL2497 and AGAIN I’ve had WONDERFUL service in the first class cabin.  Thanks again @Delta Please share my gratitude with your crew!  #delta #deltaskymiles #travel #First 

3aeb3760-8d3e-5750-8aef-1953bd089c0d 

User_205654: if a flight booked with miles is cancelled within the cancellation window, how long does it take for miles to be refunded to my acct?
Delta: Hello, Robert. It should take about 24-48 hrs  

34d433e1-37c8-5698-b5df-4794a48a16ec 

User_208765: I’m having trouble reaching you by 800 #, but need to cancel my return flight from Cancun today.  Family emergency!
Delta: I should be able to assist you with your cancellation. One momen 



In [37]:
response = tenant_data.generate.fetch_objects(
    limit=20,
    grouped_task="What patterns are we seeing here in these issues?"
)

In [38]:
print(response.generated)

Based on the provided data, here are some patterns that can be observed:

- Many of the issues revolve around flight delays, cancellations, and issues with connecting flights, which can be frustrating for customers and often lead to requests for refunds or compensation.
- There are also several mentions of rude or unhelpful staff, which can negatively impact the customer experience and lead to complaints.
- Some customers have expressed dissatisfaction with the cleanliness and comfort of the planes, particularly in first-class or premium cabins, which can detract from the overall travel experience.
- Issues with checking in, both online and at the airport, are also mentioned, highlighting the importance of efficient and effective check-in processes.
- Pricing discrepancies and issues with booking flights online have also been raised, indicating a need for transparency and accuracy in the booking process.
- On the positive side, there are also mentions of excellent customer service from

In [42]:
from weaviate.classes.query import Filter

response = tenant_data.generate.fetch_objects(
    limit=20,
    grouped_task="What patterns are we seeing here in these issues?",
    filters=Filter.by_property("text").like("*new york*")
)

In [47]:
for o in response.objects:
    print(o.uuid)
    print(o.properties["text"], "\n")

13030d41-cd5a-526b-b5b4-d25ac5247231
User_203779: what happens if you have a basic economy ticket and flight is delayed or cancelled?
Delta: We would get you on the next available flight. A Basic Economy ticket would only inhibit voluntary changes. *CBK
User_203779: So in other words you are still covered in case of a delay or travel waiver situation like the one today in New York and Denver?
Delta: The only changes you cannot make would be voluntary changes. If there were any delays or cancellations you would be re-accommodated. *CBK 

142acc3b-40b9-5c78-ba6a-e2139dd8132e
User_178663: ...like this. Please let us know what can be done about this. Thank you #flight407 from Los Angeles to New York City
Delta: Jordan, in order for assistance to be provided please follow via DM. Thanks. *TDL https://t.co/6iDGBJAc2m 

1df0bd40-46ce-54a5-872b-592df262bd52
User_232871: I need a drink. I do not like small planes. @Delta #veryclaustrophobic
Delta: Sorry about the uneasiness Stephanie! We'll get

In [40]:
print(response.generated)

The issues presented in these texts seem to revolve around luggage problems and flight experiences with Delta Airlines. 

The first issue highlights a customer's complaint about their missing baggage, which was left in Atlanta despite the customer arriving in Chicago. This could indicate a potential issue with baggage handling or transfer between connecting flights. 

The second issue, on the other hand, showcases positive feedback from a customer who had a pleasant flight experience with Delta, praising their on-time performance despite the weather conditions. 

Overall, these issues seem to reflect the range of experiences customers can have with an airline, from baggage-related problems to timely and satisfactory flights. It's important for airlines to address these issues promptly and efficiently to ensure customer satisfaction and loyalty.


### Tenant management




### Demo application

- Outside of the notebook
